In [1]:
from s1t1_fingerprint import *

dataset_path = "/home/simo/dl/comp2021/samsung_s1t1/data/split_0/data_train.txt"

atom_dict = defaultdict(lambda: len(atom_dict))
bond_dict = defaultdict(lambda: len(bond_dict))
fingerprint_dict = {}
edge_dict = defaultdict(lambda: len(edge_dict))


RAD = 1
bonds = {BT.SINGLE: 0, BT.DOUBLE: 1, BT.TRIPLE: 2, BT.AROMATIC: 3}
data_list = []

with open(dataset_path, "r") as f:
    data = f.read().strip().split("\n")

    for i, line in enumerate(data):
        line = line.split(" ")
        smiles = line[0]
        #print(smiles)
        
        mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
        N = mol.GetNumAtoms()

        # Create node data
        atoms = create_atoms_n_update(mol, atom_dict)

        i_jbond_dict = create_ijbonddict(mol, bond_dict)
        fingerprints = count_fingerprints(
            RAD, atoms, i_jbond_dict, fingerprint_dict, edge_dict
        )

        

In [2]:
atom_dict

defaultdict(<function __main__.<lambda>()>,
            {'C': 0,
             'N': 1,
             'O': 2,
             ('C', 'aromatic'): 3,
             ('O', 'aromatic'): 4,
             ('N', 'aromatic'): 5,
             'H': 6,
             'S': 7,
             'Cl': 8,
             ('S', 'aromatic'): 9,
             'F': 10,
             'Br': 11,
             'Si': 12,
             'I': 13,
             'P': 14,
             'B': 15,
             ('P', 'aromatic'): 16})

In [3]:
# use topk Elements for actual fingerprint : if not in dict, use the mol itself.
fingerprint_dict_sorted = sorted(fingerprint_dict.items(), key=lambda x: x[1])
fingerprint_dict_topk = {k: v for k, v in fingerprint_dict_sorted[-20:]}
fingerprint_dict_topk = {K[0] : idx for idx, K in enumerate(fingerprint_dict_topk.items())}

In [4]:
atom_dict
atom_ndict = {k : v for k, v in atom_dict.items()}
atom_ndict


{'C': 0,
 'N': 1,
 'O': 2,
 ('C', 'aromatic'): 3,
 ('O', 'aromatic'): 4,
 ('N', 'aromatic'): 5,
 'H': 6,
 'S': 7,
 'Cl': 8,
 ('S', 'aromatic'): 9,
 'F': 10,
 'Br': 11,
 'Si': 12,
 'I': 13,
 'P': 14,
 'B': 15,
 ('P', 'aromatic'): 16}

In [5]:
# pickle dump.

import pickle

with open('atom_dict.pickle', 'wb') as handle:
    pickle.dump(atom_ndict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('fingerprint_dict_topk.pickle', 'wb') as handle:
    pickle.dump(fingerprint_dict_topk, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [6]:
# try import and load.
with open("atom_dict.pickle", 'rb') as handle:
    atom_dict = pickle.load(handle)

with open("fingerprint_dict_topk.pickle", 'rb') as handle:
    fingerprint_dict = pickle.load(handle)

atom_dict, fingerprint_dict



({'C': 0,
  'N': 1,
  'O': 2,
  ('C', 'aromatic'): 3,
  ('O', 'aromatic'): 4,
  ('N', 'aromatic'): 5,
  'H': 6,
  'S': 7,
  'Cl': 8,
  ('S', 'aromatic'): 9,
  'F': 10,
  'Br': 11,
  'Si': 12,
  'I': 13,
  'P': 14,
  'B': 15,
  ('P', 'aromatic'): 16},
 {(0, ((3, 0), (6, 0), (6, 0), (6, 0))): 0,
  (3, ((3, 2), (5, 2), (6, 0))): 1,
  (3, ((3, 2), (3, 2), (3, 2))): 2,
  (2, ((0, 0), (3, 0))): 3,
  (1, ((0, 0), (0, 0), (0, 0))): 4,
  (1, ((0, 0), (0, 0), (6, 0))): 5,
  (0, ((0, 0), (2, 0), (6, 0), (6, 0))): 6,
  (0, ((0, 0), (1, 0), (2, 1))): 7,
  (3, ((2, 0), (3, 2), (3, 2))): 8,
  (5, ((3, 2), (3, 2))): 9,
  (3, ((1, 0), (3, 2), (3, 2))): 10,
  (2, ((0, 1),)): 11,
  (0, ((0, 0), (1, 0), (6, 0), (6, 0))): 12,
  (6, ((1, 0),)): 13,
  (0, ((0, 0), (6, 0), (6, 0), (6, 0))): 14,
  (3, ((0, 0), (3, 2), (3, 2))): 15,
  (0, ((0, 0), (0, 0), (6, 0), (6, 0))): 16,
  (3, ((3, 2), (3, 2), (6, 0))): 17,
  (6, ((3, 0),)): 18,
  (6, ((0, 0),)): 19})